In [9]:
# Import the dependencies.
import pandas as pd
from citipy import citipy
import numpy as np
import os
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
# Import the datetime module from the datetime library.
from datetime import datetime

In [10]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 

In [11]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [12]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

605

In [13]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key + "&lang=en"

In [14]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
   
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        WeatherDescriptionList  = city_weather["weather"]
        WeatherDescription = WeatherDescriptionList[0]["description"]
        try:
            Rain1HR = "{:.1f}".format(city_weather["rain"]["1h"] / 25.4)
        except:
            Rain1HR = "{:.1f}".format(0)
        try:
            Snow1HR = "{:.1f}".format(city_weather["snow"]["1h"] / 25.4)
        except:
            Snow1HR = "{:.1f}".format(0)
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description" : WeatherDescription,
                          "Rain (Inches)" : Rain1HR,
                          "Snow (Inches)" : Snow1HR,
                          "Date" : city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

 of Set 3 | butaritari
Processing Record 48 of Set 3 | aklavik
Processing Record 49 of Set 3 | pandan
Processing Record 50 of Set 3 | surt
Processing Record 1 of Set 4 | sri aman
Processing Record 2 of Set 4 | lompoc
Processing Record 3 of Set 4 | salina cruz
Processing Record 4 of Set 4 | parana
Processing Record 5 of Set 4 | kalmar
Processing Record 6 of Set 4 | ahipara
Processing Record 7 of Set 4 | palmer
Processing Record 8 of Set 4 | bilibino
Processing Record 9 of Set 4 | wendo
Processing Record 10 of Set 4 | cayenne
Processing Record 11 of Set 4 | hadgaon
Processing Record 12 of Set 4 | olafsvik
Processing Record 13 of Set 4 | sorland
Processing Record 14 of Set 4 | hasaki
Processing Record 15 of Set 4 | enid
Processing Record 16 of Set 4 | saint-augustin
Processing Record 17 of Set 4 | kolyvan
Processing Record 18 of Set 4 | dikson
Processing Record 19 of Set 4 | montepuez
Processing Record 20 of Set 4 | kompaniyivka
Processing Record 21 of Set 4 | kahului
Processing Record 22

In [15]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches),Date
0,Mataura,NZ,-46.19,168.86,41.38,91,100,5.75,overcast clouds,0.0,0.0,2020-07-09 18:44:51
1,Hambantota,LK,6.12,81.12,78.80,100,40,3.36,scattered clouds,0.0,0.0,2020-07-09 18:44:51
2,Laguna,US,38.42,-121.42,84.99,43,0,1.01,clear sky,0.0,0.0,2020-07-09 18:44:51
3,Lebu,CL,-37.62,-73.65,53.69,93,100,24.94,light rain,0.0,0.0,2020-07-09 18:44:51
4,Mahebourg,MU,-20.41,57.70,71.60,68,75,17.22,light intensity shower rain,0.0,0.0,2020-07-09 18:44:52
5,Avarua,CK,-21.21,-159.78,75.20,73,20,10.29,few clouds,0.0,0.0,2020-07-09 18:44:52
6,Georgetown,MY,5.41,100.34,80.60,94,20,3.87,few clouds,0.0,0.0,2020-07-09 18:44:52
7,Punta Arenas,CL,-53.15,-70.92,37.40,77,99,23.04,overcast clouds,0.0,0.0,2020-07-09 18:40:07
8,Rafai,CF,4.95,23.92,70.45,95,55,1.45,broken clouds,0.0,0.0,2020-07-09 18:44:52
9,Vao,NC,-22.67,167.48,67.33,73,0,23.20,clear sky,0.0,0.0,2020-07-09 18:35:06


In [16]:
# Create the output file (CSV).
output_data_file = os.path.join("weather_data","WeatherPy_database.csv")
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")